In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import os
import time  # For tracking time
from tensorflow.keras.utils import to_categorical

# Organize the dataset directory path
dataset_path = 'E:/Harini/SLIIT/4 year/RP/images' 

# Function to prepare the image for MobileNetV2
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize image to 224x224
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)  # Preprocess for MobileNetV2
    return img_array

# Custom data generator class for loading images
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, batch_size, target_size, class_mode):
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.class_mode = class_mode
        self.classes = os.listdir(directory)  # Get the list of class names (subfolders)
        self.class_indices = {c: idx for idx, c in enumerate(self.classes)}  # Mapping class names to labels
        self.filenames = []

        # Gather all image filenames
        for class_name in self.classes:
            class_dir = os.path.join(directory, class_name)
            for filename in os.listdir(class_dir):
                self.filenames.append((class_name, filename))

    def __len__(self):
        # Return the number of batches per epoch
        return int(np.floor(len(self.filenames) / self.batch_size))

    def __getitem__(self, index):
        # Get a batch of filenames
        batch_filenames = self.filenames[index * self.batch_size: (index + 1) * self.batch_size]

        images = []
        labels = []

        start_time = time.time()  # Start timing the batch process

        # Process each image in the batch
        for class_name, filename in batch_filenames:
            img_path = os.path.join(self.directory, class_name, filename)
            img_array = prepare_image(img_path)  # Prepare the image for MobileNetV2

            images.append(img_array)
            labels.append(self.class_indices[class_name])  # Convert class name to index

        # Convert lists to numpy arrays
        images = np.vstack(images)
        labels = to_categorical(labels, num_classes=4)  # Convert labels to one-hot encoded format

        batch_time = time.time() - start_time  # Calculate batch processing time
        print(f"Batch {index + 1}: Processing Time = {batch_time:.2f} seconds")

        return images, labels


# Build the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Add output layer
output = Dense(4, activation='softmax')(x)  # 4 classes: not_mature, partially_mature, mature, rotten

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base MobileNetV2 layers (optional, can be fine-tuned later)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

#  Train the model using the custom generator
batch_size = 32
train_generator = CustomDataGenerator(dataset_path, batch_size=batch_size, target_size=(224, 224), class_mode='categorical')

start_train_time = time.time()  # Start timing training
model.fit(train_generator, epochs=10)
end_train_time = time.time()  # End timing training

print(f"Total Training Time: {end_train_time - start_train_time:.2f} seconds")

# Save the model
model.save('papaya_maturity_model.h5')
print("Model training complete and saved!")

# Predicting on a single test image
def predict_image(img_path):
    # Prepare the test image
    img_array = prepare_image(img_path)

    # Start timing inference
    start_infer_time = time.time()
    prediction = model.predict(img_array)
    end_infer_time = time.time()

    # Get the class with the highest probability
    class_idx = np.argmax(prediction, axis=1)

    # Class labels (modify based on your dataset)
    class_labels = ['mature', 'not_mature', 'partially_mature', 'rotten']

    # Display the result
    predicted_class = class_labels[class_idx[0]]
    confidence = np.max(prediction) * 100  # Get the confidence of the prediction
    infer_time = end_infer_time - start_infer_time

    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")
    print(f"Inference Time: {infer_time:.4f} seconds")

# Example of testing with a new image
test_image_path = 'E:/Harini/SLIIT/4 year/RP/images/mature/Mature_004.jpg'  # Replace with the actual path to your testing image
predict_image(test_image_path)
